In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import sys
sys.path.append("/project/")

from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, PercentFormatter
import seaborn as sns
import matplotlib.lines as mlines


from utils import get_df

In [3]:
ALGO_LIST = [
    "mcpg_me",
    "dcg_me",
    "pga_me",
    "me",
    "memes",
]

In [4]:
results_dir = Path("fig2/output/")
EPISODE_LENGTH = 250
df = get_df(results_dir, EPISODE_LENGTH)
#df['algo'] = df.apply(filter, axis=1)
df = df[df["algo"].isin(ALGO_LIST)]
df = df[df["num_evaluations"] <= 1_005_000]

ant_uni_250
walker2d_uni_250
ant_omni_250
hopper_uni_250
anttrap_omni_250


In [5]:
idx = df.groupby(["env", "algo", "run"])["iteration"].idxmax()
df_last_iteration = df.loc[idx]
df = df_last_iteration[['env', 'algo', 'time', 'qd_score', 'batch_size']]

df[['qd_score', 'time']] = df.groupby(['env', 'algo'])[['qd_score', 'time']].transform(
    lambda x: (x - x.min()) / (x.max() - x.min())
)

df = df.groupby(['env', 'algo', 'batch_size'])[['qd_score', 'time']].median().reset_index()

df['time'] = 1 - df['time']

df['scalability_score'] = df['qd_score'] * df['time']

df = df.groupby(['algo', 'batch_size'])[['scalability_score']].mean().reset_index()

/tmp/ipykernel_3565987/2536057986.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['qd_score', 'time']] = df.groupby(['env', 'algo'])[['qd_score', 'time']].transform(


In [6]:
df

,algo,batch_size,scalability_score
0,dcg_me,256,0.006617
1,dcg_me,512,0.421822
2,dcg_me,1024,0.686505
3,dcg_me,2048,0.676569
4,dcg_me,4096,0.433361
5,mcpg_me,256,0.013911
6,mcpg_me,512,0.209318
7,mcpg_me,1024,0.383933
8,mcpg_me,2048,0.521025
9,mcpg_me,4096,0.598087


In [7]:
df = df.loc[df.groupby(['algo'])['scalability_score'].idxmax()]

In [8]:
df

,algo,batch_size,scalability_score
2,dcg_me,1024,0.686505
9,mcpg_me,4096,0.598087
14,me,4096,0.581361
17,memes,64,0.326350
21,pga_me,1024,0.458365
